In [ ]:
using HTTP, CSV, DataFrames, StringEncodings, Plots

In [ ]:
POS_CSV_URL = "https://www.mhlw.go.jp/content/pcr_positive_daily.csv"

In [ ]:
TST_CSV_URL = "https://www.mhlw.go.jp/content/pcr_tested_daily.csv"

In [ ]:
SVR_CSV_URL = "https://www.mhlw.go.jp/content/severe_daily.csv"

In [ ]:
DTH_CSV_URL = "https://www.mhlw.go.jp/content/death_total.csv"

In [ ]:
DL_DIR = "."

In [ ]:
HTTP.download(POS_CSV_URL, DL_DIR)

In [ ]:
HTTP.download(TST_CSV_URL, DL_DIR)

In [ ]:
HTTP.download(SVR_CSV_URL, DL_DIR)

In [ ]:
HTTP.download(DTH_CSV_URL, DL_DIR)

In [ ]:
function get_csv_file(url)
    return string(DL_DIR, "/", last(split(url, '/')))
end

In [ ]:
POS_CSV_FILE = get_csv_file(POS_CSV_URL)

In [ ]:
TST_CSV_FILE = get_csv_file(TST_CSV_URL)

In [ ]:
SVR_CSV_FILE = get_csv_file(SVR_CSV_URL)

In [ ]:
DTH_CSV_FILE = get_csv_file(DTH_CSV_URL)

In [ ]:
ENCODING = enc"UTF-8"

In [ ]:
RATE_LIMIT = 1.0

In [ ]:
pos_sum = 0

In [ ]:
function read_csv(file_path)
    df = CSV.File(open(read, file_path, ENCODING)) |> DataFrame
    return df
end

In [ ]:
df_pos = read_csv(POS_CSV_FILE)

In [ ]:
df_tst = read_csv(TST_CSV_FILE)

In [ ]:
df_svr = read_csv(SVR_CSV_FILE)

In [ ]:
df_dth = read_csv(DTH_CSV_FILE)

In [ ]:
sum = 0

In [ ]:
function get_pos_sum(x)
    global pos_sum += x
    return pos_sum
end

In [ ]:
df_pos[:, "累積感染者数"] = map(x -> get_pos_sum(x), df_pos[:, "PCR 検査陽性者数(単日)"])

In [ ]:
df_tmp = innerjoin(df_pos, df_tst, on = :日付)

In [ ]:
df_tmp2 = innerjoin(df_tmp, df_svr, on = :日付)

In [ ]:
df = innerjoin(df_tmp2, df_dth, on = :日付)

In [ ]:
df[:, "陽性率"] = df[:, "PCR 検査陽性者数(単日)"] ./ df[:, "PCR 検査実施件数(単日)"]

In [ ]:
function get_rate_limited(rate)
    global RATE_LIMIT
    if rate > RATE_LIMIT
        rate = RATE_LIMIT
    end
    return rate
end

In [ ]:
df[:, "陽性率"] = map(x -> get_rate_limited(x), df[:, "陽性率"])

In [ ]:
df[:, "致死率"] = df[:, "死亡者数"] ./ df[:, "累積感染者数"]

In [ ]:
println(df)

In [ ]:
plot(df[:, "日付"], df[:, "PCR 検査陽性者数(単日)"], st=:bar, size=(1024, 500))

In [ ]:
plot!(df[:, "日付"], df[:, "重症者数"], st=:bar, size=(1024, 500))

In [ ]:
plot(df[:, "日付"], df[:, "累積感染者数"], size=(1024, 500))

In [ ]:
plot(df[:, "日付"], df[:, "死亡者数"], size=(1024, 500))

In [ ]:
plot(df[:, "日付"], df[:, "致死率"], size=(1024, 500))

In [ ]:
plot(df[:, "日付"], df[:, "陽性率"], size=(1024, 500))